In [67]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph
import pandas as pd

sparql = SPARQLWrapper("http://sparql.uniprot.org/sparql")

In [68]:
patric = pd.read_table('patricmd5.tbl')
patric = patric.dropna()
patric

,genome.genome_id,feature.patric_id,feature.aa_sequence_md5
0,1313.13767,fig|1313.13767.peg.1898,6ef34467dbca4ed42b15c48b4d40c02f
1,1313.13767,fig|1313.13767.peg.224,a41951fdca3a19bec422891ea67a4acb
2,1313.13767,fig|1313.13767.peg.1834,b9b71ef0ce41207da47388d3b072bcd3
3,1313.13767,fig|1313.13767.peg.183,35bd757df6f26624ef3c857e77bd4b04
4,1313.13767,fig|1313.13767.peg.1674,3530a1c52e0625d0bdf929f535d1e4f1
...,...,...,...
3165,1313.13767,fig|1313.13767.peg.1091,55c10219b92713d26354afe478a4a5cc
3166,1313.13767,fig|1313.13767.peg.2167,ee178afde37c4649e34d69b0b25c7d06
3169,1313.13767,fig|1313.13767.peg.2084,fe9686a98321862e396a6d6006edd544
3172,1313.13767,fig|1313.13767.peg.295,07e29e9e33a01d6dad9ca0a12d01193b


In [69]:
def toString(md5array):
    s = '('
    for md5 in md5array:
        s = s + '"' + md5 + '",'
    s = s[:-1] + ')'
    return s

In [70]:
patricmd5 = patric['feature.aa_sequence_md5'].values
patricmd5


array(['6ef34467dbca4ed42b15c48b4d40c02f',
       'a41951fdca3a19bec422891ea67a4acb',
       'b9b71ef0ce41207da47388d3b072bcd3', ...,
       'fe9686a98321862e396a6d6006edd544',
       '07e29e9e33a01d6dad9ca0a12d01193b',
       '9431396bb91f5f50cbc7bb3b320cd442'], dtype=object)

In [71]:
search = f"""
    PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX taxon: <http://purl.uniprot.org/taxonomy/>

    SELECT DISTINCT
        (CONCAT(SUBSTR(STR(?protein), 33)) AS ?uniprot)
        ?md5 
        (CONCAT(SUBSTR(STR(?taxon), 34)) AS ?species)
    WHERE {{
        ?protein a up:Protein ;
            up:organism ?taxon ;
            up:sequence ?sequence .
        ?sequence up:md5Checksum ?md5 .
        FILTER (?md5 in ("6ef34467dbca4ed42b15c48b4d40c02f", "a41951fdca3a19bec422891ea67a4acb", "b9b71ef0ce41207da47388d3b072bcd3"))
        FILTER (?taxon in (taxon:1313))
    }}
"""
sparql.setQuery(search) 

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

uniprot = pd.DataFrame(columns=('md5', 'uniprotID', 'species'))
for row in results['results']['bindings']:
    uniprot.loc[len(uniprot.index)] = [
        row['md5']['value'], 
        row['uniprot']['value'],
        int(row['species']['value'])
    ]
uniprot

,md5,uniprotID,species
0,f3ff2a045ea7197ca9d3054784c1df48,A0A1V5ALL7,1811726
1,88965364a93628d8571ebbb3668d0d09,A0A6B0STJ6,2073113
2,6a34a875a14305f87ed26fc7d0da9eb4,M0CBQ0,1230457
3,2206f29ebaf2f16209fe432d64a786e3,A0A0Q0ZQB0,1735326
4,2206f29ebaf2f16209fe432d64a786e3,A0A0Q0ZQB0,1735326
...,...,...,...
995,9f8c6496d22705bcf554485bb0a32a64,B7NLF9,585057
996,8ef7436be78c48a9efc00128823b4243,B7UJX2,574521
997,78cab15cdf7d9ddf175e3962f23b2baf,B7UJX3,574521
998,80db15c7b2a888abdb4a5e29d217cf18,C0PZQ9,476213


In [72]:
merged = pd.merge(uniprot, patric, left_on = 'md5', right_on = 'feature.aa_sequence_md5', how = 'inner')
merged

,md5,uniprotID,species,genome.genome_id,feature.patric_id,feature.aa_sequence_md5


In [73]:
crosslinks = merged[['md5', 'uniprotID', 'feature.patric_id']]
crosslinks

,md5,uniprotID,feature.patric_id
